In [1]:
from kaggle_secrets import UserSecretsClient

In [2]:
user_secrets = UserSecretsClient()
token = user_secrets.get_secret("GITHUB_KEY")

In [3]:
!git clone https://{token}@github.com/shibha20/llama-3-finetune.git
    

Cloning into 'llama-3-finetune'...
remote: Enumerating objects: 132, done.
remote: Counting objects: 100% (132/132), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 132 (delta 65), reused 74 (delta 19), pack-reused 0
Receiving objects: 100% (132/132), 27.96 MiB | 19.94 MiB/s, done.
Resolving deltas: 100% (65/65), done.


In [ ]:
# %cd /kaggle/working/llama-3-finetune

In [ ]:
# !git checkout -b Update-Eval-Notebook

In [ ]:
# !git pull origin Update-Eval-Notebook

In [4]:
%%capture
!mamba install --force-reinstall aiohttp -y
!pip install -U "xformers<0.0.26" --index-url https://download.pytorch.org/whl/cu121
!pip install "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"

# Temporary fix for https://github.com/huggingface/datasets/issues/6753
!pip install datasets==2.16.0 fsspec==2023.10.0 gcsfs==2023.10.0

In [5]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.0 MB/s eta 0:00:00


In [6]:
from datasets import load_dataset, load_metric
import sys
import os
sys.path.append('/kaggle/working/llama-3-finetune/src/')
from data.data_sampler import sample_data
from unsloth import FastLanguageModel
import torch
import evaluate
import re

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2024-06-30 02:06:10.277663: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-30 02:06:10.277767: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-30 02:06:10.419389: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
split_ratio = 0.2
seed = 42

In [8]:
dataset_name = "CohereForAI/aya_dataset"
dataset_type = "aya"

In [9]:
lang_multi = "Nepali"
_, aya_nepali_test = sample_data(dataset_name, dataset_type, split_ratio, seed, output_dir=None, lang_multi=lang_multi)

Generating train split:   0%|          | 0/202362 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1750 [00:00<?, ? examples/s]

Filter:   0%|          | 0/202362 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1750 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4002 [00:00<?, ? examples/s]

In [10]:
aya_nepali_test = aya_nepali_test.select(range(200)) 
aya_nepali_test

Dataset({
    features: ['inputs', 'targets', 'language', 'language_code', 'annotation_type', 'user_id'],
    num_rows: 200
})

In [11]:
lang_multi = "Hindi"
_, aya_hindi_test = sample_data(dataset_name, dataset_type, split_ratio, seed, output_dir=None, lang_multi=lang_multi)

Filter:   0%|          | 0/202362 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1750 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1153 [00:00<?, ? examples/s]

In [12]:
aya_hindi_test = aya_hindi_test.select(range(200))
aya_hindi_test

Dataset({
    features: ['inputs', 'targets', 'language', 'language_code', 'annotation_type', 'user_id'],
    num_rows: 200
})

In [ ]:
dataset_name = "Saugatkafley/alpaca-nepali-sft"
_, alpaca_nepali_test = sample_data(dataset_name, "alpaca", split_ratio, seed, output_dir=None, lang_multi=None)

In [ ]:
alpaca_nepali_test = alpaca_nepali_test.select(range(200))


In [ ]:
dataset_name = "iamshnoo/alpaca-cleaned-hindi"
_, alpaca_hindi_test = sample_data(dataset_name, "alpaca", split_ratio, seed, output_dir=None, lang_multi=None)

In [ ]:
alpaca_hindi_test = alpaca_hindi_test.select(range(200))


In [ ]:
alpaca_prompt_hindi = """Below is an instruction in Hindi that describes a task, paired with an input also in hindi that provides further context. Write a response in Hindi that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [ ]:
alpaca_prompt_nepali = """Below is an instruction in Nepali that describes a task, paired with an input also in nepali that provides further context. Write a response in Nepali that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [13]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.6
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.2+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [14]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

def run_inference(dataset, prompt, is_alpaca_format=True):
    bleu_metric = evaluate.load("bleu")
    rouge_metric = evaluate.load("rouge")

    def update_row(row, is_alpaca_format=is_alpaca_format):
        if is_alpaca_format:
            instruction = row["instruction"]
            input_text = row["input"]
            prompt_text = prompt.format(instruction, input_text, "")
            target_text = row["output"]
        else:
            inputs = row["inputs"]
            prompt_text = prompt.format(inputs, "")
            target_text = row["targets"]

        inputs = tokenizer([prompt_text], return_tensors="pt").to("cuda")
        outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)
        generated_text = tokenizer.batch_decode(outputs)[0]
        response_pattern = r'### Response:\s*(.*?)\s*(?=### Explanation:|<\|end_of_text\|>|$)'
        match = re.search(response_pattern, generated_text, re.DOTALL)

        if match:
            response = match.group(1).strip()
        else:
            response = "Response not found in the model output."
#         bleu_score = bleu_metric.compute(predictions=[response], references=[target_text])["bleu"]
#         rouge_score = rouge_metric.compute(predictions=[response], references=[target_text])["rouge1"]
        
        return {
            "generated_text": generated_text,
            "model_response": response,
#             "bleu_score": bleu_score,
#             "rouge_score": rouge_score,
        }
    
    updated_dataset = dataset.map(update_row)
    return updated_dataset

In [15]:
aya_prompt_nepali = """Below is an input in Nepali that is a request or context. Write a response to the instruction or complete the context in given input in Nepali.

### Input:
{}

### Response:
{}"""

In [16]:
aya_prompt_hindi = """Below is an input in Hindi that is a request or context. Write a response to the instruction or complete the context in given input in Hindi.

### Input:
{}

### Response:
{}"""

In [17]:
!pip install rouge_score

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=df2d27d4b571777de7da69002d9761e0cb3877d650a328f7ef928c603d10771c
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
alpaca_hindi_test= run_inference(alpaca_hindi_test, alpaca_prompt_hindi, is_alpaca_format=True)
alpaca_nepali_test = run_inference(alpaca_nepali_test, alpaca_prompt_nepali, is_alpaca_format=True)

In [25]:
import pandas as pd
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_colwidth', None)  # Show full column content


In [ ]:
alpaca_nepali_test = pd.DataFrame(alpaca_nepali_test)

In [ ]:
#parse through generated text to find the actual response of the model 
def find_matches(text):
    pattern = r'(?:Response:.*?(?=\b\w+:$)|(?<=Response:).*?(?=\b\w+:|$))'
    matches = re.findall(pattern, text, re.DOTALL)
    return matches

In [ ]:
# Apply the function to create a new column 'extracted_text'
alpaca_nepali_test['extracted_text'] = alpaca_nepali_test['generated_text'].apply(lambda x: '\n'.join(find_matches(x)))


In [ ]:
nep_pattern = re.compile(r'[\u0900-\u097F]$')


In [ ]:
# Function to check if string ends with Nepali Devanagari letter
def ends_with_nepali_letter(text):
    return bool(nep_pattern.search(text))




In [ ]:
# Apply the function to filter rows
incomplete_responses_alpaca_nep = alpaca_nepali_test[alpaca_nepali_test['extracted_text'].apply(ends_with_nepali_letter)]

In [ ]:
empty_responses_alpaca_nep = alpaca_nepali_test[alpaca_nepali_test['extracted_text']=="\n<|end_of_text|>"]

In [ ]:
print("Result of Inference on Alpaca Nepali Test")
print("Total Responses - Alpaca Nepali Test: ", len(alpaca_nepali_test))
print("Incomplete Responses - Alpaca Nepali Test: ", len(incomplete_responses_alpaca_nep))
print("Empty Responses - Alpaca Nepali Test: ", len(empty_responses_alpaca_nep))

In [ ]:
alpaca_hindi_test = pd.DataFrame(alpaca_hindi_test)

In [ ]:
alpaca_hindi_test['extracted_text'] = alpaca_hindi_test['generated_text'].apply(lambda x: '\n'.join(find_matches(x)))


In [ ]:
hin_pattern = re.compile(r'[^\u0900-\u097F]*[\u0900-\u097F]$')

In [ ]:
# Function to check if string ends with hindi Devanagari letter
def ends_with_hindi_letter(text):
    return bool(hin_pattern.search(text))


In [ ]:
# Apply the function to filter rows
incomplete_responses_alpaca_hindi = alpaca_hindi_test[alpaca_hindi_test['extracted_text'].apply(ends_with_hindi_letter)]

In [ ]:
empty_responses_alpaca_hin = alpaca_hindi_test[alpaca_hindi_test['extracted_text']=="\n<|end_of_text|>"]

In [ ]:
print("Result of Inference on Alpaca Hindi Test")
print("Total Responses - Alpaca Hindi Test: ", len(alpaca_hindi_test))
print("Incomplete Responses - Alpaca Hindi Test: ", len(incomplete_responses_alpaca_hindi))
print("Empty Responses - Alpaca Hindi Test: ", len(empty_responses_alpaca_hin))

In [18]:
aya_hindi_test = run_inference(aya_hindi_test, aya_prompt_nepali, is_alpaca_format=False)


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

In [26]:
aya_hindi_test = pd.DataFrame(aya_hindi_test)


In [ ]:
aya_hindi_test['extracted_text'] = aya_hindi_test['generated_text'].apply(lambda x: '\n'.join(find_matches(x)))


In [32]:
hin_pattern = re.compile(r'[^\u0900-\u097F]*[\u0900-\u097F]$')

In [33]:
# Function to check if string ends with hindi Devanagari letter
def ends_with_hindi_letter(text):
    return bool(hin_pattern.search(text))


In [34]:
incomplete_responses_aya_hindi = aya_hindi_test[aya_hindi_test['extracted_text'].apply(ends_with_hindi_letter)]

In [35]:
empty_responses_aya_hin = aya_hindi_test[aya_hindi_test['extracted_text']=="\n<|end_of_text|>"]

In [36]:
print("Result of Inference on Aya Hindi Test")
print("Total Responses - Aya Hindi Test: ", len(aya_hindi_test))
print("Incomplete Responses - Aya Hindi Test: ", len(incomplete_responses_aya_hindi))
print("Empty Responses - Aya Hindi Test: ", len(empty_responses_aya_hin))

Result of Inference on Aya Hindi Test
Total Responses - Aya Hindi Test:  200
Incomplete Responses - Aya Hindi Test:  75
Empty Responses - Aya Hindi Test:  1


In [19]:
aya_nepali_test = run_inference(aya_nepali_test, aya_prompt_nepali, is_alpaca_format=False)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

In [38]:
aya_nepali_test = pd.DataFrame(aya_nepali_test)


In [39]:
aya_nepali_test['extracted_text'] = aya_nepali_test['generated_text'].apply(lambda x: '\n'.join(find_matches(x)))


In [41]:
nep_pattern = re.compile(r'[\u0900-\u097F]$')


In [43]:
# Function to check if string ends with Nepali Devanagari letter
def ends_with_nepali_letter(text):
    return bool(nep_pattern.search(text))

In [44]:
incomplete_responses_aya_nep = aya_nepali_test[aya_nepali_test['extracted_text'].apply(ends_with_nepali_letter)]

In [45]:
empty_responses_aya_nep = aya_nepali_test[aya_nepali_test['extracted_text']=="\n<|end_of_text|>"]

In [46]:
print("Result of Inference on Aya Nepali Test")
print("Total Responses - Aya Nepali Test: ", len(aya_nepali_test))
print("Incomplete Responses - Aya Nepali Test: ", len(incomplete_responses_aya_nep))
print("Empty Responses - Aya Nepali Test: ", len(empty_responses_aya_nep))

Result of Inference on Aya Nepali Test
Total Responses - Aya Nepali Test:  200
Incomplete Responses - Aya Nepali Test:  53
Empty Responses - Aya Nepali Test:  2


In [ ]:
aya_nepali_test.to_csv("/kaggle/working/llama-3-finetune/infer_out/aya_nepali_test.csv")
aya_hindi_test.to_csv("/kaggle/working/llama-3-finetune/infer_out/aya_hindi_test.csv")
alpaca_nepali_test.to_csv("/kaggle/working/llama-3-finetune/infer_out/alpaca_nepali_test.csv")
alpaca_hindi_test.to_csv("/kaggle/working/llama-3-finetune/infer_out/alpaca_hindi_test.csv")